In [369]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [370]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [371]:
import pandas as pd
import os, sys
import datetime
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [4]:
          ### CONCAT FILES IN CSVs FOLDER ###

# files = [file_path for file_path in os.listdir('CSVs') if file_path.endswith('.csv')]

# df = pd.read_csv(os.path.join('CSVs', files[0]))

# for csv_path in files[1:]:
#     df_to_add = pd.read_csv(os.path.join('CSVs', csv_path))
#     df = pd.concat([df,df_to_add])
    
    
# df.reset_index(drop=True, inplace=True)

#df.to_csv('Kick(2021-2022).csv')

In [509]:
# COLs = ['country','location','category', 'blurb', 'slug', 'backers_count', 
#        'created_at','goal','launched_at','deadline','converted_pledged_amount',
#         'state', 'currency', 'staff_pick', 'static_usd_rate','usd_type']
    

# def wrangle(CSV_PATH):
#     df = pd.read_csv(CSV_PATH, usecols= COLs)
#     df.dropna(inplace=True)

#     #Reorder columns
#     df = df[['country','location','category', 'blurb', 'slug', 'backers_count', 
#         'created_at','launched_at','deadline','converted_pledged_amount','goal',
#         'state', 'currency',
#         'staff_pick', 'static_usd_rate','usd_type']]
    
#     # Dropping rows where state is = to 'live' or 'canceled'
#     df = df[df['state'].isin(['successful','failed'])]
    
#     # Convert str to dicts (category and location columns)
#     convert_to_dicts = lambda x: json.loads(x)
#     df['category'] = df['category'].apply(convert_to_dicts)
#     df['location'] = df['location'].apply(convert_to_dicts)
    
#     # Exctract name from dicts (category and location columns)
#     extract_name = lambda x: x['name']
#     df['category'] = df['category'].apply(extract_name)
#     df['location'] = df['location'].apply(extract_name)
    
#     # Timestamps to Dates
#     convert = lambda x: datetime.datetime.fromtimestamp(x)
#     df['created_at'] = df['created_at'].apply(convert)
#     df['launched_at'] = df['launched_at'].apply(convert)
#     df['deadline'] = df['deadline'].apply(convert)
      
#     # Rename
#     df.columns = ['country','city_name','category', 'description', 'project_name', 'n_backers', 
#        'created_at','launch_date','deadline','pledged_$','goal',
#         'state', 'currency',
#           'staff_pick', 'usd_rate','usd_type']
    
#     # New Features 
#     df['launch_month_number_of_the_year'] = [i.month for i in df['launch_date']]
#     df['campaign_duration_in_days'] = [i.days for i in duration]
    
#     # Drop Cols
#     df.drop(columns=['created_at','launch_date','deadline','usd_type', 'usd_rate','staff_pick', 'n_backers','pledged_$'], inplace=True)
    
#     # Removes Dash from (project_name column)
#     dash_remover = lambda x: x.replace('-',' ')
#     df['project_name'] = df['project_name'].apply(dash_remover)
    
#     return df

# df = wrangle('Kick(2021-2022).csv')

In [514]:
df = pd.read_csv('Kick(2021-2022)_wrangled.zip',compression = "zip")

In [515]:
df.head()

,Unnamed: 0,country,city_name,category,description,project_name,goal,state,currency,launch_month_number_of_the_year,campaign_duration_in_days
0,0,US,Burlington,Glass,I make these adorable planters myself. Custo...,i would like to produce and sell my lightbulb ...,2000.0,failed,USD,8,30
1,1,US,Vineyard,Events,"The Utah Valley Food Fest is a sampling event,...",utah valley food fest 2019,3000.0,successful,USD,7,16
2,2,BE,Antwerp,Comedy,Newly found cabaret project Bunny Burlesk Even...,bbe cabaret burlesque show,300.0,successful,EUR,10,60
3,3,US,Oxnard,Children's Books,A children's book about a bear that discovers ...,you are loved a bear odd fruit origin story,4500.0,successful,USD,12,21
4,4,US,New York,Musical,What happens when the queen of darkness decide...,wish a mystical magical murderful musical,10000.0,successful,USD,7,31


In [512]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208215 entries, 0 to 223288
Data columns (total 10 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   country                          208215 non-null  object 
 1   city_name                        208215 non-null  object 
 2   category                         208215 non-null  object 
 3   description                      208215 non-null  object 
 4   project_name                     208215 non-null  object 
 5   goal                             208215 non-null  float64
 6   state                            208215 non-null  object 
 7   currency                         208215 non-null  object 
 8   launch_month_number_of_the_year  208215 non-null  int64  
 9   campaign_duration_in_days        208215 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 17.5+ MB


In [496]:
df.head()

,country,city_name,category,description,project_name,n_backers,pledged_$,goal,state,currency,staff_pick,launch_week_of_the_year,campaign_duration_in_days
0,US,Burlington,Glass,I make these adorable planters myself. Custo...,i would like to produce and sell my lightbulb ...,2,31,2000.0,failed,USD,False,34,30
1,US,Vineyard,Events,"The Utah Valley Food Fest is a sampling event,...",utah valley food fest 2019,188,3367,3000.0,successful,USD,False,31,16
2,BE,Antwerp,Comedy,Newly found cabaret project Bunny Burlesk Even...,bbe cabaret burlesque show,12,380,300.0,successful,EUR,False,40,60
3,US,Oxnard,Children's Books,A children's book about a bear that discovers ...,you are loved a bear odd fruit origin story,89,4839,4500.0,successful,USD,True,53,21
4,US,New York,Musical,What happens when the queen of darkness decide...,wish a mystical magical murderful musical,30,10703,10000.0,successful,USD,False,27,31


In [464]:
df[df['state']=='successful']['staff_pick'].value_counts(normalize=True)

False    0.784991
True     0.215009
Name: staff_pick, dtype: float64

In [465]:
df[df['state']=='failed']['staff_pick'].value_counts(normalize=True)

False    0.970687
True     0.029313
Name: staff_pick, dtype: float64

In [396]:
test_df = df.head()

In [448]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [457]:
tokens = tokenizer.encode("so so, it was ok", return_tensors='pt')

In [458]:
result = model(tokens)

In [459]:
result.logits

tensor([[-1.4170,  0.8113,  2.6103,  0.3364, -1.9903]],
       grad_fn=<AddmmBackward>)

In [460]:
int(torch.argmax(result.logits))+1

3